# Data Pipeline - ΔG values

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-2.7-Required-Modules" data-toc-modified-id="Python-2.7-Required-Modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python 2.7 Required Modules</a></span></li><li><span><a href="#Input-Data" data-toc-modified-id="Input-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Input Data</a></span></li><li><span><a href="#Extract-data" data-toc-modified-id="Extract-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract data</a></span></li></ul></div>

## Python 2.7 Required Modules

In [ ]:
import json
import pandas as pd

## Input Data

'reactions.json' - Go to https://vmh.uni.lu/_api/reactions/?format=json&page_size=99999, save as 'reactions.json'

'reacdeltags.json' - Go to https://vmh.uni.lu/_api/reacdeltags/?format=json&page_size=99999, save as 'reacdeltags.json'

## Extract data

Load reaction information from vmh.uni.lu API json file

In [ ]:
# load data
data = json.load(open('input_data/reactions.json'))

# extract reaction id's and names
reactions = {}
for i in range(len(data['results'])):
    if data['results'][i]['isHuman'] == True:
        reactions[data['results'][i]['rxn_id']] = str(data['results'][i]['abbreviation'])

Load ΔG information from vmh.uni.lu API json file

In [ ]:
# load data
data = json.load(open('input_data/reacdeltags.json'))

# extract reaction id's and deltag info
deltag = {}
for i in range(len(data['results'])):
    deltag[data['results'][i]['rxn']] = {}
    deltag[data['results'][i]['rxn']]['standard'] = float(data['results'][i]['standardEnergy'])
    deltag[data['results'][i]['rxn']]['uncertainty'] = float(data['results'][i]['uncertainty'])
    deltag[data['results'][i]['rxn']]['transformed'] = float(data['results'][i]['transformedEnergy'])
    deltag[data['results'][i]['rxn']]['min_transformed'] = float(data['results'][i]['minimumTransformed'])
    deltag[data['results'][i]['rxn']]['max_transformed'] = float(data['results'][i]['maximumTransformed'])

List of reactions to not include

In [ ]:
not_include = ['TRDR']

Output results

In [ ]:
df = pd.DataFrame(columns=['REACTION','STANDARD [kJ/mol]','UNCERTAINTY [kJ/mol]','TRANSFORMED [kJ/mol]','MIN TRANSFORMED [kJ/mol]','MAX TRANSFORMED [kJ/mol]'])
for i in reactions:
    if i in deltag:
        if reactions[i] not in not_include:
            df.loc[i] = [reactions[i],deltag[i]['standard'],deltag[i]['uncertainty'],deltag[i]['transformed'],deltag[i]['min_transformed'],deltag[i]['max_transformed']]
        
df.to_csv('deltag.csv',index=None)